In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tslearn.clustering import TimeSeriesKMeans
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, LSTM, Input, Flatten, Reshape
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
import numpy as np
import os
import pandas as pd
import pywt

In [15]:
epoch_num = 5
sequence_length = 8  # Number of time steps to consider

# Define file and directory names
file_name = '0_61938.csv'
data_dir = 'ohlc_data'
# parent_dir = 'forexPredictor'
# repo = 'Repos_git'
# repo_dir = 'Make_Money_with_Tensorflow_2.0'
# Get the absolute base directory dynamically
base_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))  # Move up one level

# Construct the full file path in an OS-independent way
# data_path = os.path.join(base_dir, repo, repo_dir, parent_dir, data_dir, file_name)
data_path = os.path.join(base_dir, data_dir, file_name)

# Check if the file exists before using it
if os.path.exists(data_path):
    print(f"✅ File found: {data_path}")
else:
    print(f"❌ Error: File not found at {data_path}")


ucols=['Open', 'High', 'Low', 'Close']
data = pd.read_csv(data_path, usecols=ucols)
data.reset_index(drop=True, inplace=True)
data.head()


✅ File found: D:\Repos_git\Make_Money_with_Tensorflow_2.0\forexPredictor\ohlc_data\0_61938.csv


,Open,High,Low,Close
0,1.20997,1.21089,1.20966,1.20999
1,1.20481,1.20569,1.20479,1.20538
2,1.20537,1.20574,1.20341,1.20553
3,1.20556,1.20689,1.20442,1.20469
4,1.20468,1.20599,1.20380,1.20573


In [16]:
df = data
# Convert DataFrame to NumPy array
data = df.values

In [18]:
data.shape

(61938, 4)

In [19]:
# Normalize data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data = scaler.fit_transform(df)

# Reshape for LSTM input
data = data.reshape((data.shape[0], data.shape[1], 1))

In [20]:
data.shape

(61938, 4, 1)

In [21]:
def apply_dwt(data):
    coeffs = [pywt.dwt(sample, 'haar') for sample in data]
    cA, cD = zip(*coeffs)  # Approximation and Detail coefficients
    return np.array(cA)  # Using only approximation coefficients

# Define Autoencoder with LSTM for better feature extraction
def create_autoencoder(input_shape):
    input_layer = Input(shape=input_shape)
    x = LSTM(64, return_sequences=True)(input_layer)
    x = LSTM(32, return_sequences=False)(x)
    encoded = Dense(16, activation='relu')(x)

    decoded = Dense(32, activation='relu')(encoded)
    decoded = Dense(input_shape[0], activation='sigmoid')(decoded)

    autoencoder = Model(input_layer, decoded)
    encoder = Model(input_layer, encoded)

    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder, encoder


In [22]:
# Apply DWT
transformed_data = apply_dwt(data)

# Train Autoencoder
autoencoder, encoder = create_autoencoder(transformed_data.shape[1:])
autoencoder.fit(transformed_data, transformed_data, epochs=5, batch_size=32, verbose=1)


Epoch 1/5
1936/1936 [==============================] - 8s 3ms/step - loss: 0.0088
Epoch 2/5
1936/1936 [==============================] - 6s 3ms/step - loss: 0.0059
Epoch 3/5
1936/1936 [==============================] - 6s 3ms/step - loss: 0.0059
Epoch 4/5
1936/1936 [==============================] - 6s 3ms/step - loss: 0.0059
Epoch 5/5
1936/1936 [==============================] - 6s 3ms/step - loss: 0.0059


In [23]:
autoencoder.save("encoder_model.h5")

C:\Users\Teacher\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Extract Features
features = encoder.predict(transformed_data)

# Apply TimeSeriesKMeans Clustering
num_clusters = 3  # Buy (2), Sell (1), Hold (0)
tskmeans = TimeSeriesKMeans(n_clusters=num_clusters, metric="dtw", random_state=42)
clusters = tskmeans.fit_predict(features)


1936/1936 [==============================] - 3s 1ms/step


In [ ]:
# Convert clusters to one-hot encoding
clusters_one_hot = tf.keras.utils.to_categorical(clusters, num_clusters)

# Split data into train and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(features, clusters_one_hot, test_size=0.2, random_state=42)


In [ ]:
# Ensure input shape consistency
input_shape = features.shape[1:]  # Extract feature dimension
classifier_input = Input(shape=input_shape)
classifier_output = Dense(num_clusters, activation="softmax")(classifier_input)
classifier_output.shape

In [ ]:
# Create and compile classifier
final_model = Model(classifier_input, classifier_output)
final_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

epoch_num = 5  # Define the number of epochs
final_model.fit(X_train, y_train, epochs=epoch_num, batch_size=5, validation_data=(X_test, y_test))

final_model.save("trading_model_dwt_autoEnc.h5")

# Evaluate model on test data
loss, accuracy = final_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Confusion Matrix & Classification Report
y_pred = final_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

print("Confusion Matrix:")
print(confusion_matrix(y_test_classes, y_pred_classes))
print("Classification Report:")
print(classification_report(y_test_classes, y_pred_classes))

In [42]:
loaded_model = load_model("trading_model_dwt_autoEnc.h5")


C:\Users\Teacher\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# documentation: https://www.mql5.com/en/docs/integration/python_metatrader5

import MetaTrader5 as mt  # pip install MetaTrader5
import pandas as pd  # pip install pandas
from datetime import datetime, timedelta
import os

# start the platform with initialize()
mt.initialize()

# login to Trade Account with login()
# make sure that trade server is enabled in MT5 client terminal

login = 165905041
password = 'iIeElL0176_'
server = 'XMGlobal-MT5 2'

mt.login(login, password, server)



In [ ]:
# Define symbol and timeframe
symbol = "EURUSD"
timeframe = mt.TIMEFRAME_H1

# Get data for the last 48 hours
end_time = datetime.now()
start_time = end_time - timedelta(hours=1000)

# Retrieve OHLC data
ohlc_data = pd.DataFrame(mt.copy_rates_range(symbol, timeframe, start_time, end_time))

# Convert time column to datetime format
ohlc_data['time'] = pd.to_datetime(ohlc_data['time'], unit='s')


ohlc_data.rename(columns={'open':'Open', 'high':'High', 
                        'low':'Low', 'close':'Close', 'tick_volume':'Volume'}, inplace=True)
columns_to_drop = ['time', 'spread', 'real_volume', 'Volume']
ohlc_data.drop(columns=[col for col in columns_to_drop if col in ohlc_data.columns], inplace=True)



In [ ]:
ohlc_data

In [ ]:
new_data = ohlc_data

In [ ]:
# Load the saved scaler (Assuming it's saved from training to ensure consistency)
# import joblib
# scaler = joblib.load("scaler.pkl")

# Load new data
new_data = new_data[['Open', 'High', 'Low', 'Close']].values  # Extract OHLC values

# Normalize using the same MinMaxScaler used during training
ohlc_data_scaled = scaler.transform(new_data)  # Use transform, NOT fit_transform

# Reshape for LSTM input
data = ohlc_data_scaled.reshape((new_data.shape[0], new_data.shape[1], 1))


In [81]:
data.shape

(713, 4, 1)

In [ ]:
# Apply DWT
transformed_data = apply_dwt(data)
print("Transformed Data Shape:", transformed_data.shape)


In [ ]:
# Load trained encoder and classifier
encoder = tf.keras.models.load_model("encoder_model.h5")
classifier = tf.keras.models.load_model("trading_model_dwt_autoEnc.h5")

# Extract features using the trained encoder
features = encoder.predict(transformed_data)

# Make predictions
predictions = classifier.predict(features)
predicted_classes = np.argmax(predictions, axis=1)  # Convert probabilities to class labels

# Map predictions to labels
labels = {0: "Hold", 1: "Sell", 2: "Buy"}
predicted_labels = [labels[p] for p in predicted_classes]

# Align predictions with the original DataFrame
new_data_df = pd.DataFrame(new_data, columns=['Open', 'High', 'Low', 'Close'])
new_data_df["Prediction"] = predicted_labels  # Add predicted actions to DataFrame

# Save the results
new_data_df.to_csv("predicted_results.csv", index=False)

# Print first 10 rows with predictions
print(new_data_df.head(10))


In [ ]:
import numpy as np
import pandas as pd
import pywt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tslearn.clustering import TimeSeriesKMeans
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

# Load Data
data_path = "stock_data.csv"  # Update with your file path
df = pd.read_csv(data_path, usecols=['Open', 'High', 'Low', 'Close'])

# Normalize Data
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df)

def apply_dwt(data):
    coeffs = [pywt.dwt(sample, 'haar') for sample in data]
    cA, cD = zip(*coeffs)
    return np.array(cA)  # Using only approximation coefficients

# Apply DWT
transformed_data = apply_dwt(df_scaled)
transformed_data = transformed_data.reshape((transformed_data.shape[0], transformed_data.shape[1], 1))

# Define LSTM Autoencoder
def create_autoencoder(input_shape):
    input_layer = Input(shape=input_shape)
    x = LSTM(64, return_sequences=True)(input_layer)
    x = LSTM(32, return_sequences=False)(x)
    encoded = Dense(16, activation='relu')(x)
    
    decoded = Dense(32, activation='relu')(encoded)
    decoded = Dense(input_shape[0], activation='sigmoid')(decoded)
    
    autoencoder = Model(input_layer, decoded)
    encoder = Model(input_layer, encoded)
    
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder, encoder

# Train Autoencoder
autoencoder, encoder = create_autoencoder(transformed_data.shape[1:])
autoencoder.fit(transformed_data, transformed_data, epochs=5, batch_size=32, verbose=1)

# Extract Features
features = encoder.predict(transformed_data)

# Clustering with TimeSeriesKMeans
num_clusters = 3  # Buy (2), Sell (1), Hold (0)
tskmeans = TimeSeriesKMeans(n_clusters=num_clusters, metric="dtw", random_state=42)
clusters = tskmeans.fit_predict(features)
clusters_one_hot = tf.keras.utils.to_categorical(clusters, num_clusters)

# Split Data for Training
X_train, X_test, y_train, y_test = train_test_split(features, clusters_one_hot, test_size=0.2, random_state=42)

# Define Classifier
input_shape = features.shape[1:]  # Extract feature dimension
classifier_input = Input(shape=input_shape)
classifier_output = Dense(num_clusters, activation="softmax")(classifier_input)

classifier = Model(classifier_input, classifier_output)
classifier.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train Classifier
classifier.fit(X_train, y_train, epochs=5, batch_size=5, validation_data=(X_test, y_test))

# Evaluate Model
loss, accuracy = classifier.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Confusion Matrix & Classification Report
y_pred = classifier.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

print("Confusion Matrix:")
print(confusion_matrix(y_test_classes, y_pred_classes))
print("Classification Report:")
print(classification_report(y_test_classes, y_pred_classes))

# Predict on New Data
def predict_new_data(new_data):
    new_data_scaled = scaler.transform(new_data)
    transformed_new_data = apply_dwt(new_data_scaled)
    transformed_new_data = transformed_new_data.reshape((transformed_new_data.shape[0], transformed_new_data.shape[1], 1))
    features_new = encoder.predict(transformed_new_data)
    predictions = classifier.predict(features_new)
    predicted_classes = np.argmax(predictions, axis=1)
    labels = {0: "Hold", 1: "Sell", 2: "Buy"}
    return [labels[p] for p in predicted_classes]

# Example Usage
new_data = pd.read_csv("new_stock_data.csv", usecols=['Open', 'High', 'Low', 'Close']).values
predictions = predict_new_data(new_data)
print(predictions)
